In [1]:
import pandas as pd
import numpy as np
from statistics import stdev
from sklearn.cluster import KMeans as kmeans
pd.set_option('display.max_colwidth', None) #so tables are not truncated
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv("C:\\Users\\akash\\Documents\\Election Stats\\ANES 2020 2\\edited data\\anes_timeseries_2020_csv_20210719.csv")

C:\Users\akash\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (15,17,18,19,21,22,23,25,26,27,29,30,31,33,34,35,37,38,1508,1509) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df = df.drop(df.columns[[15,17,18,19,21,22,23,25,26,27,29,30,31,33,34,35,37,38,1508,1509]], axis=1) 
#import file and get rid of some columns with bad data types

In [4]:
extra_data = {'V200010a': 'weight'}

#replace (V\d\d\d\d\d\d[a-z]) (.+\n?.+)(?=\nQuestion) with '$1': '$2',\n

demographics = {'V201435' : 'PRE: WHAT IS PRESENT RELIGION OF R', 
                'V201462' : 'PRE: RELIGIOUS IDENTIFICATION', 
                'V201508' : 'PRE: MARITAL STATUS',
               'V201511x' : 'PRE: SUMMARY: RESPONDENT 5 CATEGORY LEVEL OF EDUCATION',
               #'V201575' : 'PRE: WHERE R GREW UP', sample sizes too small
               'V201601' : 'PRE: SEXUAL ORIENTATION OF R [REVISED]',
               'V201617x' : 'PRE: SUMMARY: TOTAL (FAMILY) INCOME',
               'V202110x' : 'PRE-POST: SUMMARY: 2020 PRESIDENTIAL VOTE',
               'V201600' : 'PRE: WHAT IS YOUR (R) SEX? [REVISED]',
               'V201549x' : 'PRE: SUMMARY: R SELF-IDENTIFIED RACE/ETHNICITY',
               'V201021' : 'PRE: FOR WHICH CANDIDATE DID R VOTE IN PRESIDENTIAL PRIMARY',
               'V201200' : 'PRE: 7PT SCALE LIBERAL-CONSERVATIVE SELFPLACEMENT',
               'V201507x' : 'Age'}

media_sources = {
    'V201630a' : 'PRE: MENTION: TV PROG - THE LATE SHOW WITH STEPHEN COLBERT',
    'V201630c' : 'PRE: MENTION: TV PROG - TUCKER CARLSON TONIGHT (FOX)',
    'V201630d' : 'PRE: MENTION: TV PROG - THE RACHEL MADDOW SHOW (MSNBC)',
    'V201630e' : 'PRE: MENTION: TV PROG - THE LAST WORD WITH LAWRENCE O’DONNELL (MSNBC)',
    'V201630f': 'PRE: MENTION: TV PROG - THE FIVE (FOX)',
    'V201630g': 'PRE: MENTION: TV PROG - THE INGRAHAM ANGLE (FOX)',
    'V201630h': 'PRE: MENTION: TV PROG - THE STORY WITH MARTHA MACCALLUM (FOX)',
    'V201630i': 'PRE: MENTION: TV PROG - THE LEAD WITH JAKE TAPPER (CNN)',
    'V201630j': 'PRE: MENTION: TV PROG - ANDERSON COOPER 360 (CNN)',
    'V201630k': 'PRE: MENTION: TV PROG - SPECIAL REPORT WITH BRET BAIER (FOX)',
    'V201630m': 'PRE: MENTION: TV PROG - CBS EVENING NEWS WITH NORAH O’DONNELL',
    'V201630n': 'PRE: MENTION: TV PROG - ABC WORLD NEWS TONIGHT',
    'V201630p': 'PRE: MENTION: TV PROG - NBC NIGHTLY NEWS WITH LESTER HOLT',
    'V201630q': 'PRE: MENTION: TV PROG - CUOMO PRIME TIME (CNN)',
    'V201631c': 'PRE: MENTION: TV PROG - 60 MINUTES (CBS)',
    'V201631d': 'PRE: MENTION: TV PROG - 20/20 (ABC)',
    'V201631e': 'PRE: MENTION: TV PROG - DATELINE NBC',
    'V201631f': 'PRE: MENTION: TV PROG - FACE THE NATION (CBS)',
    'V201631g': 'PRE: MENTION: TV PROG - MEET THE PRESS (NBC)',
    'V201631h': 'PRE: MENTION: TV PROG - CBS THIS MORNING',
    'V201631i': 'PRE: MENTION: TV PROG - GOOD MORNING AMERICA (ABC)',
    'V201631j': 'PRE: MENTION: TV PROG - TODAY (NBC)',
    'V201631k': 'PRE: MENTION: TV PROG - FOX & FRIENDS (FOX)',
    'V201631m': 'PRE: MENTION: TV PROG - MORNING JOE (MSNBC)',
    'V201631n': 'PRE: MENTION: TV PROG - ALL IN WITH CHRIS HAYES (MSNBC)',
    'V201631p': 'PRE: MENTION: TV PROG - THE 11TH HOUR WITH BRIAN WILLIAMS (MSNBC)',
    'V201631q': 'PRE: MENTION: TV PROG - PBS NEWSHOUR',
    'V201631r': 'PRE: MENTION: TV PROG - SATURDAY NIGHT LIVE (NBC)',
    'V201633a': 'PRE: MENTION: RADIO PROG - RUSH LIMBAUGH',
    'V201633b': 'PRE: MENTION: RADIO PROG - THE SEAN HANNITY SHOW',
    'V201633c': 'PRE: MENTION: RADIO PROG - MARKETPLACE (NPR)',
    'V201633d': 'PRE: MENTION: RADIO PROG - ALL THINGS CONSIDERED (NPR)',
    'V201633e': 'PRE: MENTION: RADIO PROG - THE DAVE RAMSEY SHOW',
    'V201633f': 'PRE: MENTION: RADIO PROG - MORNING EDITION (NPR)',
    'V201633g': 'PRE: MENTION: RADIO PROG - THE MARK LEVIN SHOW',
    'V201633h': 'PRE: MENTION: RADIO PROG - THE GLENN BECK PROGRAM',
    'V201633i': 'PRE: MENTION: RADIO PROG - THE MIKE GALLAGHER SHOW',
    'V201633j': 'PRE: MENTION: RADIO PROG - THE HUGH HEWITT SHOW',
    'V201633k': 'PRE: MENTION: RADIO PROG - THE SAVAGE NATION (MICHAEL SAVAGE)',
    'V201633m': 'PRE: MENTION: RADIO PROG - THE DANA SHOW',
    'V201633n': 'PRE: MENTION: RADIO PROG - THE THOM HARTMANN PROGRAM',
    'V201633p': 'PRE: MENTION: RADIO PROG - THE JIM BOHANNON SHOW',
    'V201633q': 'PRE: MENTION: RADIO PROG - FRESH AIR (NPR)',
    'V201634a': 'PRE: MENTION: WEBSITE - YAHOO NEWS (WWW.YAHOO.COM/NEWS)',
    'V201634b': 'PRE: MENTION: WEBSITE - CNN.COM',
    'V201634c': 'PRE: MENTION: WEBSITE - HUFFINGTON POST (WWW.HUFFINGTONPOST.COM)',
    'V201634d': 'PRE: MENTION: WEBSITE - NEW YORK TIMES (NYTIMES.COM)',
    'V201634f': 'PRE: MENTION: WEBSITE - FOX NEWS (WWW.FOXNEWS.COM)',
    'V201634g': 'PRE: MENTION: WEBSITE - WASHINGTON POST (WASHINGTONPOST.COM)',
    'V201634h': 'PRE: MENTION: WEBSITE - THE GUARDIAN (THEGUARDIAN.COM)',
    'V201634i': 'PRE: MENTION: WEBSITE - USA TODAY (USATODAY.COM)',
    'V201634j': 'PRE: MENTION: WEBSITE - BBC NEWS (NEWS.BBC.CO.UK)',
    'V201634k': 'PRE: MENTION: WEBSITE - NPR NEWS (NPR.ORG)',
    'V201634m': 'PRE: MENTION: WEBSITE - DAILY CALLER (DAILYCALLER.COM)',
    'V201634n': 'PRE: MENTION: WEBSITE - BLOOMBERG (BLOOMBERG.COM)',
    'V201634p': 'PRE: MENTION: WEBSITE - BUZZFEED (BUZZFEED.COM)',
    'V201634q': 'PRE: MENTION: WEBSITE - NBC NEWS (WWW.NBCNEWS.COM)',
    'V201635a': 'PRE: MENTION: PRINTED NEWSPAPER - THE NEW YORK TIMES',
    'V201635b': 'PRE: MENTION: PRINTED NEWSPAPER - USA TODAY',
    'V201635c': 'PRE: MENTION: PRINTED NEWSPAPER - THE WALL STREET JOURNAL',
    'V201635d': 'PRE: MENTION: PRINTED NEWSPAPER - THE WASHINGTON POST',
    'V201636a': 'PRE: MENTION: ONLINE NEWSPAPER - WWW.NYTIMES.COM (NEW YORK TIMES)',
    'V201636b': 'PRE: MENTION: ONLINE NEWSPAPER - WWW.USATODAY.COM (USA TODAY)',
    'V201636c': 'PRE: MENTION: ONLINE NEWSPAPER - ONLINE.WSJ.COM (WALL STREET JOURNAL)',
    'V201636d': 'PRE: MENTION: ONLINE NEWSPAPER - WWW.WASHINGTONPOST.COM (WASH POST)'
}


ideology_questions = {'V201234' : 'PRE: GOVERNMENT RUN BY A FEW BIG INTERESTS OR FOR BENEFIT OF ALL',
'V201235' : 'PRE: DOES GOVERNMENT WASTE MUCH TAX MONEY',
'V201236' : 'PRE: HOW MANY IN GOVERNMENT ARE CORRUPT',
'V201237' : 'PRE: HOW OFTEN CAN PEOPLE BE TRUSTED',
'V201238' : 'PRE: ELECTIONS MAKE GOVERNMENT PAY ATTENTION',
'V201246' : 'PRE: 7PT SCALE SPENDING & SERVICES: SELFPLACEMENT',
'V201249' : 'PRE: 7PT SCALE DEFENSE SPENDING: SELF-PLACEMENT',
'V201252' : 'PRE: 7PT SCALE GOV-PRIVATE MEDICAL INSURANCE SCALE: SELF-PLACEMENT',
'V201255' : 'PRE: 7PT SCALE GUARANTEED JOB-INCOME SCALE: SELF-PLACEMENT',
'V201258' : 'PRE: 7PT SCALE GOV ASSISTANCE TO BLACKS SCALE: SELF-PLACEMENT',
'V201262' : 'PRE: 7PT SCALE ENVIRONMENT-BUSINESS TRADEOFF: SELF-PLACEMENT',
'V201302x' : 'PRE: FEDERAL BUDGET SPENDING: SOCIAL SECURITY',
'V201305x' : 'PRE: SUMMARY: FEDERAL BUDGET SPENDING: PUBLIC SCHOOLS',
'V201308x' : 'PRE: SUMMARY: FEDERAL BUDGET SPENDING: TIGHTENING BORDER SECURITY',
'V201311x' : 'PRE: SUMMARY: FEDERAL BUDGET SPENDING: DEALING WITH CRIME',
'V201314x' : 'PRE: SUMMARY: FEDERAL BUDGET SPENDING: WELFARE PROGRAMS',
'V201317x' : 'PRE: SUMMARY: FEDERAL BUDGET SPENDING: BUILDING AND REPAIRING HIGHWAYS',
'V201320x' : 'PRE: SUMMARY: FEDERAL BUDGET SPENDING: AID TO THE POOR',
'V201323x' : 'PRE: SUMMARY: FEDERAL BUDGET SPENDING: PROTECTING THE ENVIRONMENT',
'V201336' : 'RE: STD ABORTION: SELF-PLACEMENT',
'V201345x' : 'PRE: SUMMARY: R FAVOR/OPPOSE DEATH PENALTY',
'V201349x' : 'PRE: SUMMARY: COUNTRY WOULD BE BETTER OFF IF WE JUST STAYED HOME',
'V201350' : 'PRE: FORCE TO SOLVE INTERNATIONAL PROBLEMS',
'V201356x' : 'PRE: SUMMARY: FAVOR/OPPOSE VOTE BY MAIL',
'V201359x' : 'PRE: SUMMARY: FAVOR/OPPOSE REQUIRING ID WHEN VOTING',
'V201362x' : 'PRE: SUMMARY: FAVOR/OPPOSE ALLOWING FELONS TO VOTE',
'V201366' : 'PRE: HOW IMPORTANT THAT NEWS ORGANIZATIONS FREE TO CRITICIZE',
'V201367' : 'PRE: HOW IMPORTANT BRANCHES OF GOVERNMENT KEEP ONE ANOTHER FROM TOO MUCH POWER',
'V201368' : 'PRE: HOW IMPORTANT ELECTED OFFICIALS FACE SERIOUS CONSEQUENCES FOR MISCONDUCT',
'V201369' : 'PRE: HOW IMPORTANT THAT PEOPLE AGREE ON BASIC FACTS',
'V201372x' : 'PRE: SUMMARY: HELPFUL/HARMFUL IF PRES DIDN’T HAVE TO WORRY ABOUT CONGRESS/COURTS',
'V201375x' : 'PRE: SUMMARY: FAVOR OR OPPOSE RESTRICTING JOURNALIST ACCESS',
'V201376' :  'PRE: HOW CONCERNED GOVERNMENT MIGHT UNDERMINE MEDIA',
'V201377' : 'PRE: HOW MUCH TRUST IN NEWS MEDIA',
'V201378' : 'PRE: APPROPRIATE/INAPPROPRIATE PRES ASK FOREIGN COUNTRIES TO INVESTIGATE RIVALS',
'V201379' : 'PRE: PREFER GOVERNMENT OFFICIAL WHO COMPROMISES OR STICKS TO PRINCIPLES',
'V201386x' : 'PRE: SUMMARY: FAVOR OR OPPOSE HOUSE IMPEACHMENT DECISION',
'V201389x' : 'PRE: SUMMARY: FAVOR OR OPPOSE SENATE ACQUITTAL DECISION',
'V201392x' : 'PRE: SUMMARY: FEDERAL GOVERNMENT RESPONSE TO COVID-19',
'V201405x' : 'PRE: SUMMARY: REQUIRE EMPLOYERS TO OFFER PAID LEAVE TO PARENTS OF NEW CHILDREN',
'V201408x' : 'PRE: SUMMARY: SERVICES TO SAME SEX COUPLES',
'V201411x' : 'PRE: SUMMARY: TRANSGENDER POLICY',
'V201414x' : 'PRE: SUMMARY: FAVOR/OPPOSE LAWS PROTECT GAYS LESBIANS AGAINST JOB DISCRIMINATION',
'V201415' : 'PRE: SHOULD GAY AND LESBIAN COUPLES BE ALLOWED TO ADOPT',
'V201416' : 'PRE: R POSITION ON GAY MARRIAGE',
'V201417' : 'PRE: US GOVERNMENT POLICY TOWARD UNAUTHORIZED IMMIGRANTS',
'V201420x' : 'PRE: SUMMARY: FAVOR OR OPPOSE ENDING BIRTHRIGHT CITIZENSHIP',
'V201423x' : 'PRE: SUMMARY: SHOULD CHILDREN BROUGHT ILLEGALLY BE SENT BACK OR ALLOWED TO STAY',
'V201426x' : 'PRE: SUMMARY: FAVOR OR OPPOSE BUILDING A WALL ON BORDER WITH MEXICO',
'V201429' : 'PRE: BEST WAY TO DEAL WITH URBAN UNREST',
                     'V201626' : 'PRE: NEED TO BE MORE SENSITIVE TALKING OR PEOPLE TOO EASILY OFFENDED',
                     'V201639' : 'PRE: WOMEN INTERPRET INNOCENT REMARKS AS SEXIST',
                     'V201640' : 'PRE: WOMEN SEEK TO GAIN POWER BY GETTING CONTROL OVER MEN',
                     'V202212' : 'POST: [STD] PUBLIC OFFICIALS DON’T CARE WHAT PEOPLE THINK',
                     'V202213' : 'POST: [STD] HAVE NO SAY ABOUT WHAT GOVERMENT DOES',
                     'V202231x' : 'POST: SUMMARY: FAVOR/OPPOSE NEW LIMITS ON IMPORTS',
                     'V202232' : 'POST: WHAT SHOULD IMMIGRATION LEVELS BE',
                     'V202236x' : 'POST: SUMMARY: FAVOR/OPPOSE ALLOWING REFUGEES TO COME TO US',
                     'V202242x' : 'POST: SUMMARY: FAVOR/OPPOSE PROVIDING PATH TO CITIZESHIP',
                     'V202245x' : 'POST: SUMMARY: FAVOR/OPPOSE RETURNING UNAUTHORIZE IMMIGRANTS TO NATIVE COUNTRY',
                     'V202248x' : 'POST: SUMMARY: FAVOR/OPPOSE SEPARATING CHILDREN OF DETAINED IMMIGRANTS',
                     'V202252x' : 'POST: SUMMARY: FAVOR/OPPOSE PREFERENTIAL HIRING/PROMOTION OF BLACKS',
                     'V202255x' : 'POST: SUMMARY: LESS OR MORE GOVERNMENT',
                     'V202259x' : 'POST: SUMMARY: FAVOR/OPPOSE GOVERNMENT TRYING TO REDUCE INCOME INEQUALITY',
                     'V202260' : 'POST: SOCIETY SHOULD MAKE SURE EVERYONE HAS EQUAL OPPORTUNITY',
                     'V202261' : 'POST: WE’D BE BETTER OFF IF WORRIED LESS ABOUT EQUALITY',
                     'V202262' : 'POST: NOT A BIG PROBLEM IF SOME HAVE MORE CHANCE IN LIFE',
                     'V202263' : 'POST: IF PEOPLE WERE TREATED MORE FAIRLY WE WOULD HAVE FEWER PROBLEMS',
                     'V202264' : 'POST: THE WORLD IS CHANGING & WE SHOULD ADJUST VIEW OF MORAL BEHAVIOR',
                     'V202265' : 'POST: FEWER PROBLEMS IF THERE WAS MORE EMPHASIS ON TRADITIONAL FAMILY VALUES',
                     'V202270' : 'POST: BETTER IF REST OF WORLD MORE LIKE AMERICA',
                     'V202273x' : 'POST: SUMMARY: US BETTER OR WORSE THAN MOST OTHER COUNTRIES',
                     'V202279x' : 'POST: SUMMARY: PEOPLE IN RURAL AREAS HAVE TOO MUCH/TOO LITTLE INFLUENCE',
                     'V202282x' : 'POST: SUMMARY: PEOPLE IN RURAL AREAS GET TOO MUCH/TOO LITTLE RESPECT',
                     'V202290x' : 'POST: SUMMARY: BETTER/WORSE IF MAN WORKS AND WOMAN TAKES CARE OF HOME',
                     'V202291' : 'POST: DO WOMEN DEMANDING EQUALITY SEEK SPECIAL FAVORS',
                     'V202292' : 'POST: DO WOMEN COMPLAINING ABOUT DISCRIMINATION CAUSE MORE PROBLEMS',
                     'V202300' : 'POST: AGREE/DISAGREE: BLACKS SHOULD WORK THEIR WAY UP WITHOUT SPECIAL FAVORS',
                     'V202302' : 'POST: AGREE/DISAGREE: BLACKS HAVE GOTTEN LESS THAN THEY DESERVE',
                     'V202303' : 'POST: AGREE/DISAGREE: IF BLACKS TRIED HARDER THEY’D BE AS WELL OFF AS WHITES',
                     'V202304' : 'POST: OUR POLITICAL SYSTEM ONLY WORKS FOR INSIDERS WITH MONEY AND POWER',
                     'V202305' : 'POST: BECAUSE OF RICH AND POWERFUL IT’S DIFFICULT FOR THE REST TO GET AHEAD',
                     'V202308x' : 'POST: SUMMARY: TRUST ORDINARY PEOPLE/EXPERTS FOR PUBLIC POLICY',
                     'V202309' : 'POST: HOW MUCH DO PEOPLE NEED HELP FROM EXPERTS TO UNDERSTAND SCIENCE',
                     'V202310' : 'POST: HOW IMPORTANT SHOULD SCIENCE BE FOR DECISIONS ABOUT COVID-19',
                     'V202311' : 'POST: BUSINESS AND POLITICS CONTROLLED BY FEW POWERFUL PEOPLE',
                     'V202312' : 'POST: MUCH OF WHAT PEOPLE HEAR IN SCHOOLS AND MEDIA ARE LIES BY THOSE IN POWER',
                     'V202317' : 'POST: HOW MUCH OPPORTUNITY IN AMERICA FOR AVERAGE PERSON TO GET AHEAD',
                     'V202321' : 'POST: IMPORTANCE OF REDUCING DEFICIT'}
 
questions = ['Would you say the government is pretty much run by a few big interests looking out for themselves or that it is run for the benefit of all the people?',
'Do you think that people in government waste a lot of the money we pay in taxes, waste some of it, or don’t waste very much of it?',
'How many of the people running the government are corrupt?',
'Generally speaking, how often can you trust other people?',
'How much do you feel that having elections makes the government pay attention to what the people think?',
'Some people think the government should provide fewer services even in areas such as health and education in order to reduce spending. Suppose these people are at one end of a scale, at point 1. Other people feel it is important for the government to provide many more services even if it means an increase in spending. Suppose these people are at the other end, at point 7. Where would you place yourself on this scale, or haven’t you thought much about this?',
'Some people believe that we should spend much less money for defense. Suppose these people are at one end of a scale, at point 1. Others feel that defense spending should be greatly increased. Suppose these people are at the other end, at point 7. Where would you place yourself on this scale, or haven’t you thought much about this?',
'There is much concern about the rapid rise in medical and hospital costs. Some people feel there should be a government insurance plan which would cover all medical and hospital expenses for everyone. Suppose these people are at one end of a scale, at point 1. Others feel that all medical expenses should be paid by individuals through private insurance plans like Blue Cross or other company paid plans. Suppose these people are at the other end, at point 7.',
'Some people feel the government in Washington should see to it that every person has a job and a good standard of living. Suppose these people are at one end of a scale, at point 1. Others think the government should just let each person get ahead on their own. Suppose these people are at the other end, at point 7.',
'Some people feel that the government in Washington should make every effort to improve the social and economic position of blacks. Suppose these people are at one end of a scale, at point 1. Others feel that the government should not make any special effort to help blacks because they should help themselves. Suppose these people are at the other end, at point 7.',
'Some people think we need much tougher government regulations on business in order to protect the environment. Suppose these people are at one end of a scale, at point 1. Others think that current regulations to protect the environment are already too much of a burden on business. Suppose these people are at the other end, at point 7.',
'Should federal spending on Social Security be increased, decreased, or kept the same?',
'Should federal spending on public schools be increased, decreased, or kept the same?',
'Should federal spending on tightening border security to prevent illegal immigration be increased, decreased, or kept the same?',
'Should federal spending on dealing with crime be increased, decreased, or kept the same?',
'Should federal spending on welfare programs be increased, decreased, or kept the same?',
'Should federal spending on building and repairing highways be increased, decreased, or kept the same?',
'Should federal spending on aid to the poor be increased, decreased, or kept the same?',
'Should federal spending on protecting the environment be increased, decreased, or kept the same?',
'1. By law, abortion should never be permitted 2. The law should permit abortion only in case of rape, incest, or when the woman’s life is in danger 3. The law should permit abortion other than for rape/incest/danger to woman but only after need clearly established 4. By law, a woman should always be able to obtain an abortion as a matter of personal choice',
'Do you favor or oppose the death penalty for persons convicted of murder?',
'‘This country would be better off if we just stayed home and did not concern ourselves with problems in other parts of the world.’',
'How willing should the United States be to use military force to solve international problems?',
'Do you favor, oppose, or neither favor nor oppose conducting all elections by mail, instead of people voting in-person?',
'Do you favor, oppose, or neither favor nor oppose requiring all people to show a government issued photo ID when they vote?',
'Do you favor, oppose, or neither favor nor oppose allowing convicted felons to vote once they complete their sentence?',
'For the next few items, we would like to know how important you think each one is to the United States maintaining a strong democracy. First, how important is it that news organizations are free to criticize political leaders?',
'How important is it that the executive, legislative, and judicial branches of government keep one another from having too much power?',
'How important is it that elected officials face serious consequences if they engage in misconduct?',
'How important is it that people agree on basic facts even if they disagree politically?',
'Would it be helpful, harmful, or neither helpful nor harmful if U.S. presidents could work on the country’s problems without paying attention to what Congress and the courts say?',
'Do you favor, oppose, or neither favor nor oppose elected officials restricting journalists’ access to information about government decision-making?',
'How concerned are you that some people in the government today might want to undermine the news media’s ability to serve as a check on governmental power?',
'In general, how much trust and confidence do you have in the news media when it comes to reporting the news fully, accurately, and fairly?',
'In general, how appropriate or inappropriate is it for the president of the United States to ask leaders of foreign countries to investigate political rivals?',
'Would you prefer a government official who compromises to get things done, or who sticks to their principles no matter what?',
'Do you favor, oppose, or neither favor nor oppose the U.S. House of Representatives’ decision in December of last year to impeach President Trump?',
'Do you favor, oppose, or neither favor nor oppose the U.S. Senate’s decision in February to acquit President Trump of the impeachment charges and thus let him remain in office?',
'Do you think the federal government’s response to the COVID-19 outbreak earlier this year was too quick, too slow, or aboutright?',
'Do you favor, oppose, or neither favor nor oppose requiring employers to offer paid leave to parents of new children?',
'Do you think business owners who provide wedding-related services should be allowed to refuse services to same-sex couples if same-sex marriage violates their religious beliefs, or do you think business owners should be required to provide services regardless of a couple’s sexual orientation?',
'Should transgender people - that is, people who identify themselves as the sex or gender different from the one they were born as - have to use the bathrooms of the gender they were born as, or should they be allowed to use the bathrooms of their identified gender?',
'Do you favor or oppose laws to protect gays and lesbians against job discrimination?',
'Do you think gay or lesbian couples should be legally permitted to adopt children?',
'Should gay people be allowed to get married?',
'How should US treat undocumented immigrants?',
'Some people have proposed that the U.S. Constitution should be changed so that the children of unauthorized immigrants do not automatically get citizenship if they are born in this country. Do you favor, oppose, or neither favor nor oppose this proposal?',
'What should happen to immigrants who were brought to the U.S. illegally as children and have lived here for at least 10 years and graduated high school here? Should they be sent back where they came from, or should they be allowed to live and work in the United States?',
'Do you favor, oppose, or neither favor nor oppose building a wall on the U.S. border with Mexico?', 
'What is the best way to deal with the problem of urban unrest and rioting?',
            'Some people think that the way people talk needs to change with the times to be more sensitive to people from different backgrounds. Others think that this has already gone too far and many people are just too easily offended. Which is closer to your opinion?',
            'Do many women interpret innocent remarks or acts as being sexist?',
            'do women want to gain power by controlling men',
            'do public officials don’t care much what people like you think?',
            'do people like you have say in what the government does?',
            'Some people have suggested placing new limits on foreign imports in order to protect American jobs. Others say that such limits would raise consumer prices and hurt American exports. Do you favor or oppose placing new limits on imports?',
            'Do you think the number of immigrants from foreign countries who are permitted to come to the United States to live should be increased a lot, increased a little, left the same as it is now, decreased a little, or decreased a lot?',
            'Do you favor, oppose, or neither favor nor oppose allowing refugees who are fleeing war, persecution, or natural disasters in other countries to come to live in the U.S.?',
            'Do you favor, oppose, or neither favor nor oppose providing a path to citizenship for unauthorized immigrants who obey the law, pay a fine, and pass security checks?',
            'Do you favor, oppose, or neither favor nor oppose returning all unauthorized immigrants to their native countries?',
            'Do you favor, oppose, or neither favor nor oppose separating the children of detained immigrants, rather than keeping them with their parents in adult detention centers?',
            'are you for or against preferential hiring and promotion of blacks?',
            'less government better or more that govt needs to do',
            'Would it be good for society to have more government regulation, about the same amount of regulation as there is now, or less government regulation?',
            'Should our society do whatever is necessary to make sure that everyone has an equal opportunity to succeed?',
            'should we be more worried about equality?',
            'Is it a problem if some have more chances in life?',
            'Would we have fewer problems if people were treated more fairly?',
            'Should we change our views of moral behavior to fit the changing world?',
            'Would we have fewer problems if more focus on traditional family ties',
            'Would the world be better if people from other countries were more like Americans?',
            'Is the US better or worse than other countries',
            'Compared to people living in cities, do people living in small towns and rural areas have too much influence, too little influence, or about the right amount of influence on government?',
            'Do people living in small towns and rural areas get too much respect, too little respect, or about the right amount of respect from people living in cities?',
            'Is it better or worse if men work and women take care of the home',
            'When women demand equality these days, how often are they actually seeking special favors?',
            'When women complain about discrimination, how often do they cause more problems than they solve?',
            'should blacks work their way up without special favors',
            'Over the past few years, blacks have gotten less than they deserve',
            'Would blacks be as well off as whites if they worked hard enough',
            'Does the political system only work for insiders with money and power?',
            'Do rich and powerful make it harder for everyone else to get ahead?', 
            'When it comes to public policy decisions, whom do you tend to trust more: ordinary people, experts, or trust both the same?',
            'How much do ordinary people need the help of experts to understand complicated things like science and health?',
            'In general, how important should science be for making government decisions about COVID-19?',
            'Are business and politics controlled by a few powerful people?',
            'Is much of the info in school and media lies designed to benefit powerful?',
            'How much opportunity is there in America today for the average person to get ahead?', 
            'How important is it to reduce the deficit?']

#answers that represent positive values in data
pos_value_answers = ["govt run for benefit of all people",
"govt doesn't waste much tax money",
"none in govt corrupt",
"people never can be trusted",
"elections don't make govt pay attention",
"govt provide many more services",
"more defense spending",
"private healthcare",
"govt shouldn't provide guaranteed jobs & ubi",
"govt shouldnt help black people",
"govt shouldn't protect environment",
"decrease social security spending",
"decrease public school spending",
"decrease border security spending",
"decrease crime prevention spending",
"decrease welfare spending",
"decrease highway spending",
"decrease poor aid spending",
"decrease environment spending",
"abortion always allowed",
"no death penalty",
"us should not stay home",
"us shouldnt use force internationally",
"vote in person",
"no voter id laws",
"ex felons should not vote",
"important that news can criticize politicians",
"checks and balances important",
"important elected officials face consequences for misconduct",
"important people agree on basic facts",
"bad if us president could bypass courts and congress",
"do not restrict journalist access to govt info",
"concerned govt want to undermine media",
"confident in news media",
"pres shouldnt ask foreign countries to investigate rivals",
"stick to principles and not compromise",
"against trump impeachment house",
"against trump impeachment senate",
"covid response too slow",
"no paid parental leave",
"business should provide services to same sex couples",
"against trans bathroom bills",
"oppose gay antidiscrimiation law",
"gay people shouldnt be allowed to adopt children",
"no gay marriage",
"allow undocumented immigrants to become citizens",
"don't end birthright citizenship",
"children of undocumented shouldn't be deported",
"oppose border wall",
"use force to prevent urban unrest",
                    'people too easily offended',
                    'most women don\'t interpret innocent remarks or acts as being sexist.',
                    'women do not seek to gain power by controlling men',
                    'public officials care what i think',
                    'i have no say in govt',
                    'no new limits on imports',
                    'less immigrants',
                    'oppose refugees',
                    'no path to citizenship for undocumented',
                    'oppose returning undocumented to birth countries',
                    'dont separate undocumented kids from families',
                    'no preferential hiring/promotion of blacks',
                    'more govt better',
                    'govt shouldnt reduce income inequality',
                    'shouldnt make sure everyone has equal opportunity',
                    'should worry more about equality',
                    'problem if some have more chance in life',
                    'wouldnt have fewer problems if people treated more equally',
                    'shouldn\'t adjust morality to changing world',
                    'wouldnt have less problems if focused more on traditional family ties',
                    'wouldnt be better if others more like americans',
                    'US worse than other countries',
                    'rural people have too little govt influence',
                    'rural people get too little respect',
                    'worse if traditional gender roles',
                    'women asking for equality dont want special favors',
                    'women complaining about discrimination dont cause more problems',
                    'blacks cant overcome prejudice on their own',
                    'blacks havent gotten less than they deserve',
                    'blacks dont suffer prejudice because of lack of hard work',
                    'politics only works for powerful',
                    'powerful make it harder for others to get ahead',
                    'trust experts more than ordinary people',
                    'people need help from experts to understand science',
                    'science important for covid decisions',
                    'business and politics controlled by powerful few',
                    'schools and media lie to benefit powerful few',
                    'no opportunity for average person to get ahead',
                    'reducing defecit not important']

In [5]:
print(len(ideology_questions))
print(len(questions)) #should be equal

89
89


In [6]:
df.head()

,version,V200001,V160001_orig,V200002,V200003,V200004,V200005,V200006,V200007,V200008,...,V203518,V203519,V203520,V203521,V203522,V203523,V203524,V203525,V203526,V203527
0,ANES2020TimeSeries_20210719,200015,401318,3,2,3,0,-2,-2,-2,...,Thomas Jeffery 'Tom' Cole,1,-1. Inapplicable,-1,-1,82,-1. Inapplicable,-1,-1. Inapplicable,-1
1,ANES2020TimeSeries_20210719,200022,300261,3,2,3,0,4,-1,3,...,Michael Keith 'Mike' Simpson,1,-1. Inapplicable,-1,-1,-1,-1. Inapplicable,-1,-1. Inapplicable,-1
2,ANES2020TimeSeries_20210719,200039,400181,3,2,3,0,-2,-2,-2,...,Scott William Taylor,1,-1. Inapplicable,-1,-1,81,-1. Inapplicable,-1,-1. Inapplicable,-1
3,ANES2020TimeSeries_20210719,200046,300171,3,2,3,0,-2,-2,-2,...,Devin Gerald Nunes,1,-1. Inapplicable,-1,-1,81,-1. Inapplicable,-1,-1. Inapplicable,-1
4,ANES2020TimeSeries_20210719,200053,405145,3,2,3,1,-2,-2,-2,...,Kenneth R. 'Ken' Buck,1,-1. Inapplicable,-1,-1,81,-1. Inapplicable,-1,-1. Inapplicable,-1


In [7]:
columns_to_keep = ideology_questions

bad_data_type = ['V200011a', 'V200011c', 'V200011d', 'V200012a', 'V200012c', 'V200012d',
       'V200013a', 'V200013c', 'V200013d', 'V200014a', 'V200014c', 'V200014d',
       'V200015a', 'V200015c', 'V200015d', 'V200016a', 'V200016c', 'V200016d',
       'V203055', 'V203056']

weights = df['V200010a']

ideo = df.filter(columns_to_keep.keys())
ideo = ideo.rename(columns=columns_to_keep) #filters ideological questions

for col in ideo:
	ideo[col] = ideo[col].astype(int) #reformats data


column = pd.DataFrame({list(columns_to_keep.values())[i]: questions[i] for i in range(len(columns_to_keep))}, index=[1])
column2 = pd.DataFrame({list(columns_to_keep.values())[i]: pos_value_answers[i] for i in range(len(columns_to_keep))}, index=[1])
#makes questions

ideo = pd.concat([ideo.iloc[:0], column2, ideo.iloc[0:]]).reset_index(drop=True)
ideo = pd.concat([ideo.iloc[:0], column, ideo.iloc[0:]]).reset_index(drop=True)

#adds questions to dataframe


In [8]:
demo = df.filter(demographics.keys())
demo = demo.rename(columns=demographics)
#filters demographic info

for col in demo:
	demo[col] = demo[col].astype(int)

In [9]:
media = df.filter(media_sources.keys())
media = media.rename(columns=media_sources)
#filters demographic info

for col in media:
	media[col] = media[col].astype(int)

In [10]:
ideo.head()

,PRE: GOVERNMENT RUN BY A FEW BIG INTERESTS OR FOR BENEFIT OF ALL,PRE: DOES GOVERNMENT WASTE MUCH TAX MONEY,PRE: HOW MANY IN GOVERNMENT ARE CORRUPT,PRE: HOW OFTEN CAN PEOPLE BE TRUSTED,PRE: ELECTIONS MAKE GOVERNMENT PAY ATTENTION,PRE: 7PT SCALE SPENDING & SERVICES: SELFPLACEMENT,PRE: 7PT SCALE DEFENSE SPENDING: SELF-PLACEMENT,PRE: 7PT SCALE GOV-PRIVATE MEDICAL INSURANCE SCALE: SELF-PLACEMENT,PRE: 7PT SCALE GUARANTEED JOB-INCOME SCALE: SELF-PLACEMENT,PRE: 7PT SCALE GOV ASSISTANCE TO BLACKS SCALE: SELF-PLACEMENT,...,POST: AGREE/DISAGREE: IF BLACKS TRIED HARDER THEY’D BE AS WELL OFF AS WHITES,POST: OUR POLITICAL SYSTEM ONLY WORKS FOR INSIDERS WITH MONEY AND POWER,POST: BECAUSE OF RICH AND POWERFUL IT’S DIFFICULT FOR THE REST TO GET AHEAD,POST: SUMMARY: TRUST ORDINARY PEOPLE/EXPERTS FOR PUBLIC POLICY,POST: HOW MUCH DO PEOPLE NEED HELP FROM EXPERTS TO UNDERSTAND SCIENCE,POST: HOW IMPORTANT SHOULD SCIENCE BE FOR DECISIONS ABOUT COVID-19,POST: BUSINESS AND POLITICS CONTROLLED BY FEW POWERFUL PEOPLE,POST: MUCH OF WHAT PEOPLE HEAR IN SCHOOLS AND MEDIA ARE LIES BY THOSE IN POWER,POST: HOW MUCH OPPORTUNITY IN AMERICA FOR AVERAGE PERSON TO GET AHEAD,POST: IMPORTANCE OF REDUCING DEFICIT
0,Would you say the government is pretty much run by a few big interests looking out for themselves or that it is run for the benefit of all the people?,"Do you think that people in government waste a lot of the money we pay in taxes, waste some of it, or don’t waste very much of it?",How many of the people running the government are corrupt?,"Generally speaking, how often can you trust other people?",How much do you feel that having elections makes the government pay attention to what the people think?,"Some people think the government should provide fewer services even in areas such as health and education in order to reduce spending. Suppose these people are at one end of a scale, at point 1. Other people feel it is important for the government to provide many more services even if it means an increase in spending. Suppose these people are at the other end, at point 7. Where would you place yourself on this scale, or haven’t you thought much about this?","Some people believe that we should spend much less money for defense. Suppose these people are at one end of a scale, at point 1. Others feel that defense spending should be greatly increased. Suppose these people are at the other end, at point 7. Where would you place yourself on this scale, or haven’t you thought much about this?","There is much concern about the rapid rise in medical and hospital costs. Some people feel there should be a government insurance plan which would cover all medical and hospital expenses for everyone. Suppose these people are at one end of a scale, at point 1. Others feel that all medical expenses should be paid by individuals through private insurance plans like Blue Cross or other company paid plans. Suppose these people are at the other end, at point 7.","Some people feel the government in Washington should see to it that every person has a job and a good standard of living. Suppose these people are at one end of a scale, at point 1. Others think the government should just let each person get ahead on their own. Suppose these people are at the other end, at point 7.","Some people feel that the government in Washington should make every effort to improve the social and economic position of blacks. Suppose these people are at one end of a scale, at point 1. Others feel that the government should not make any special effort to help blacks because they should help themselves. Suppose these people are at the other end, at point 7.",...,Would blacks be as well off as whites if they worked hard enough,Does the political system only work for insiders with money and power?,Do rich and powerful make it harder for everyone else to get ahead?,"When it comes to public policy decisions, whom do you tend to trust more: ordinary people, experts, or trust both the same?",How much do ordi

In [11]:
import sys

avgs = []
counts = []
stdevs = []

#normalizes answers by z-scores and scales by weight
#try to make this more efficient

#prints name of feature, min and max of each row, and standard deviation

for col_index, col in enumerate(ideo):
	print(col_index, col, end= '\t')
	min_val = min(i for i in ideo[col][2:] if i > -1)
	max_val = max(i for i in ideo[col][2:] if i < 90)
	print(min_val, max_val)
	#change range to 1-7
	sum_current = 0
	count = 0
	for index, row in ideo.iterrows(): #calculate sum and set bad vals to NA
		data = row[col]
		if(isinstance(data,int)):
			if(data < 0 or data > 89):
				ideo[col][index] = 'NA'
			else:
				ideo[col][index] = ((data - min_val) / (max_val - min_val)) * (7 - 1) + 1 #normalize range to 1 to 7
				sum_current += ideo[col][index]
				count += weights[index-2]
	counts.append(count)
	if(count == 0): #avoid divide by 0, actual value doesnt matter since col is all NA
		avgs.append(0)
	else:
		avgs.append(sum_current/count)
	stdev = 0
	for index, row in ideo.iterrows(): #calculate stdev and update for average
		data = row[col]
		if(isinstance(data,int) or isinstance(data,float)):
			row[col] -= avgs[col_index]
			stdev += row[col]*row[col]
	print(stdev)
	stdev = np.sqrt(stdev / (counts[col_index] - 1))
	stdevs.append(stdev)
	for index, row in ideo.iterrows(): #normalize and multiply by weight
		data = row[col]
		if(isinstance(data,float)):
			if(row[col] == float('inf')):
				row[col] = 0
			else:
				if(stdevs[col_index] == 0):
					print(col, col_index, counts[col_index], avgs[col_index])
				row[col] /= stdevs[col_index]
		elif(data == 'NA'):
			row[col] = 0

0 PRE: GOVERNMENT RUN BY A FEW BIG INTERESTS OR FOR BENEFIT OF ALL	1 2
38595.27267027098
1 PRE: DOES GOVERNMENT WASTE MUCH TAX MONEY	1 3
20323.669063434783
2 PRE: HOW MANY IN GOVERNMENT ARE CORRUPT	1 5
14285.501321941894
3 PRE: HOW OFTEN CAN PEOPLE BE TRUSTED	1 5
15157.084022805355
4 PRE: ELECTIONS MAKE GOVERNMENT PAY ATTENTION	1 3
34703.53893316584
5 PRE: 7PT SCALE SPENDING & SERVICES: SELFPLACEMENT	1 7
22751.546981491785
6 PRE: 7PT SCALE DEFENSE SPENDING: SELF-PLACEMENT	1 7
21764.438966397887
7 PRE: 7PT SCALE GOV-PRIVATE MEDICAL INSURANCE SCALE: SELF-PLACEMENT	1 7
33042.91432008488
8 PRE: 7PT SCALE GUARANTEED JOB-INCOME SCALE: SELF-PLACEMENT	1 7
27829.717346205493
9 PRE: 7PT SCALE GOV ASSISTANCE TO BLACKS SCALE: SELF-PLACEMENT	1 7
29992.672073275207
10 PRE: 7PT SCALE ENVIRONMENT-BUSINESS TRADEOFF: SELF-PLACEMENT	1 7
28031.650149228786
11 PRE: FEDERAL BUDGET SPENDING: SOCIAL SECURITY	1 5
18493.55781597704
12 PRE: SUMMARY: FEDERAL BUDGET SPENDING: PUBLIC SCHOOLS	1 5
22895.00214840538
1

In [12]:
for idx, ques in enumerate(questions):
    print("%s\t%f\t%f\n" % (ques, avgs[idx], stdevs[idx]))
#prints average and standard deviation for each question (after normalized to 1-7 range)

Would you say the government is pretty much run by a few big interests looking out for themselves or that it is run for the benefit of all the people?	1.926486	2.169991

Do you think that people in government waste a lot of the money we pay in taxes, waste some of it, or don’t waste very much of it?	2.040533	1.569101

How many of the people running the government are corrupt?	3.928178	1.317998

Generally speaking, how often can you trust other people?	3.714249	1.353902

How much do you feel that having elections makes the government pay attention to what the people think?	3.546761	2.048465

Some people think the government should provide fewer services even in areas such as health and education in order to reduce spending. Suppose these people are at one end of a scale, at point 1. Other people feel it is important for the government to provide many more services even if it means an increase in spending. Suppose these people are at the other end, at point 7. Where would you place yours

In [13]:
data_array = ideo.values[2:]

In [14]:
data_array

array([[-0.42695372694732836, -0.6631397645842352, -1.0835963841097787,
        ..., 1.7178641791195206, -1.7621077298994516, -1.214544281615161],
       [-0.42695372694732836, -0.6631397645842352, -2.221686346831855,
        ..., 0.13507705032870326, 1.1891031749564138, 0.7975282869952507],
       [-0.42695372694732836, -0.6631397645842352, 0.05449357861229764,
        ..., -1.447710078462114, 0.20536620667112532, 0.7975282869952507],
       ...,
       [-0.42695372694732836, -0.6631397645842352, 1.192583541334374,
        ..., 0.9264706147241119, -0.7783707616141632,
        -0.2085079973099552],
       [-0.42695372694732836, -0.6631397645842352, -1.0835963841097787,
        ..., 1.7178641791195206, -1.7621077298994516,
        -0.2085079973099552],
       [-0.42695372694732836, 3.160706997702672, 1.192583541334374, ...,
        -0.6563165140667053, 1.1891031749564138, 0.7975282869952507]],
      dtype=object)

In [15]:
data_array.shape

(8280, 89)

In [16]:
num_questions = len(questions)

#computes average answers for each demographic group

def demo_averages(column):
    stats = {}
    all_vals = [i for i in sorted(list(set(demo[column]))) if i >= 0 and i <= 90] #all demographic groups
    
    for i in all_vals:
        stats[i] = [0 for i in range(num_questions)] #calculate average loadings for each demographic group
    
    sums = {i:0 for i in all_vals}
   
    for idx, row in enumerate(data_array):
        demographic_group = demo[column][idx]
        if(demographic_group not in all_vals):
            continue
        sums[demographic_group] += weights[idx]
        for component in range(num_questions):
            stats[demographic_group][component] += row[component] * weights[idx]
        
    for idx1, val_sums in enumerate(stats):
        for idx2, comp_sum in enumerate(stats[val_sums]):
            stats[all_vals[idx1]][idx2] = comp_sum / sums[all_vals[idx1]]
            
    return stats

#call function like this: demo_averages('PRE: SUMMARY: R SELF-IDENTIFIED RACE/ETHNICITY'))

In [17]:
#prints out demographic averages in neat tables

for i in demographics:
    print(demographics[i])
    x = demo_averages(demographics[i])
    for i in x:
        print(i, end = '\t')
        for idx, val in enumerate(x[i]):
            print(val * stdevs[idx] + avgs[idx], end = '\t')
        print()
    print()

PRE: WHAT IS PRESENT RELIGION OF R
1	2.046760866623448	1.9045148934584937	4.053209718690764	3.5107064710626905	3.259458950990619	3.9597767782289077	4.778691929883855	4.538835264126145	4.740798588871576	4.332551224995197	3.7765256893060077	2.8587520440571113	2.7000239147757807	2.8580477987788795	2.563659648116954	4.498347368003225	2.49572268418748	3.3426403265186537	3.119611660966627	3.547943179419523	2.950152384387745	5.323686988151072	4.27372496703134	5.0011095186100185	2.060199225181999	3.375321919496498	4.915729493133116	6.245314662825495	6.469307125577969	5.676797519700343	5.454051502429701	4.713602439164168	4.178953865943804	2.753018264523616	5.329338672297343	3.4473573826817945	4.8000874663571835	3.2466928744208907	5.256837813269432	2.6626755555303485	2.9870747132546414	3.1325035012102758	2.2926930432600123	2.906451840662107	3.256053620711811	4.204677014568579	4.1124749887534175	5.555015856796205	3.405288402926982	4.25091755053157	4.5679122358515265	3.8595381680073775	4.673378769

2	2.3525791473290045	2.1850687281517143	4.010891159056805	4.066180549244435	3.123896607589609	4.37616450355694	4.874063604771032	4.10449019165952	4.417713661131701	4.162996855037267	3.681924483804655	2.582303362994669	2.420649445792555	3.1377718466147986	2.714526548282096	4.08891035528898	2.5837583959538626	3.0764627617710083	3.0262798111259417	3.0119375125204053	3.51628880918832	4.754480681670382	3.9917790203249472	5.082211045033624	2.0203384119752976	3.0958227799951215	4.589981577486549	5.7164452449024035	6.002176518530722	5.43231145980355	4.959398802126288	4.344534306337586	4.036621521613338	2.9449194743057894	4.977817499226456	4.290966260760875	4.6242037267991165	3.5340255299764554	5.240273556134256	2.3994492841293495	2.915448590916008	2.6270249437934767	2.784470312934246	4.027916019922751	4.453025927644104	4.307918332427524	4.453642137396902	5.68876560905603	3.5219250717514674	3.8277116206591533	4.2339422852989825	3.720695062524417	4.420910764014694	2.956657617832893	3.40076481120

1	1.9387248768239174	1.9946804372215643	3.978686382639762	3.6192778645474197	3.525248724435686	4.320247891587639	4.5059411968214835	4.061731763283702	4.465986126936253	4.071528962099504	3.344580396200771	2.826170970502412	2.476056658292472	3.1794257416334584	2.7352170546999086	4.305269413660046	2.496293582839515	3.209369756862835	2.8684340424744565	3.9396351097621647	3.0838474157564737	5.113718405593919	4.333286497222642	4.565776011572539	2.346389394878974	3.2115009639769427	5.049400615728106	6.109296045780297	6.390188647893841	5.634426732479379	5.3772344908054	4.911888472504115	4.499438062692487	3.01001567792649	5.5126079626306845	3.2955154452409334	4.257412823336576	3.794729440613911	5.507578013912211	2.4464675902687603	3.713095289367436	3.5808679324277954	2.047272719831271	2.363740356407507	2.568299669464788	4.43375393899931	4.324532779167477	5.6950526529492365	3.9286031576577796	3.8630571315704247	4.3727906965864864	4.0307768206285175	4.835544822474606	2.8980131270621814	3.23220228

1	2.7083754877137745	2.270745064086539	3.820249884271425	4.291756753089924	3.6472826077113933	4.92528425299103	4.532957779438126	3.533999263165786	4.023855704952463	3.801641856366503	3.0902625223849935	2.4279460441717764	2.410268799744602	3.3571103493077175	2.727830413732436	3.6250603259229983	2.6257228509149764	2.821094260441416	2.608010129527871	3.681363409698288	2.9880267039386177	4.314992515348926	4.2501428385528515	4.249614839242135	2.689206716953391	3.063629752745861	4.591049778823319	5.426512888967733	5.745780875516263	5.139976154057779	4.495772483390899	4.474037929052979	4.312279850143717	3.3245575361217563	4.95889350949731	4.098471647563449	4.018461300517416	4.048825071007953	5.421993409853353	2.653049529384529	4.1164891696971875	3.5636384021190572	2.521963036631889	2.686807810040336	2.9708984126040274	4.248309389610521	4.5087174771703875	5.643584131212362	4.326187353412935	3.4986285438415745	4.216735198743261	3.8016240402988233	4.441921448244042	2.7662243260431074	3.034359073

1	2.5950797505714513	2.405078225144152	3.829259596540952	4.471803792052254	3.7412092647278783	5.055353069809819	4.495056422949413	3.6755113089525198	3.958785500703381	3.616419493387905	3.1830216441811543	2.4987861922085215	2.371464360326429	3.5841879162644252	2.9296558472956704	3.368557531848053	2.787161165995653	2.5861070212623356	2.4907910465871774	3.9831140480850675	3.1469899612809122	4.3180164624985125	4.12213193662696	4.350356845896677	2.868926089210071	3.3057008624661433	4.463527292675153	5.060822442497558	5.4704532713080685	4.849687329409456	4.434784685758923	4.281432191976446	4.183750817219867	3.3575747062241366	4.787343375400686	4.043421231362713	3.716377177739108	4.313343460794343	5.5299839111048	2.6952206497143174	4.4812533951156714	3.673724780041639	2.6588418267797755	2.7298645602856904	2.8344975742215053	4.316457271969609	4.356341801999982	5.474173118071089	4.24403687848919	3.361821130362514	4.050836929724161	4.073160895366788	4.488036499116587	2.7828337338071236	3.0965828

...

1	1.7405460381988003	2.2510877026624363	3.4657069985565396	3.6044464742355657	3.7105719707527194	6.177812025977691	2.673971373440903	1.8613325898759323	2.1891390280925345	1.7138701827982228	1.5477047767857492	2.1377231130406074	1.4355105937821873	5.788749430691822	4.493731784603279	1.962996651366887	2.4159201718481054	1.4732263588851393	1.312700282706556	5.180317617648168	5.512911237134626	5.290289415453141	4.866090103673437	2.7102174423402277	4.600478006346022	1.6837209654425676	6.22238070871383	6.353733938120213	6.67798962103848	6.0198241405031645	5.882036108986537	6.0407637427666945	6.026145997505684	3.912476506261531	6.434901454223462	3.2170039899643763	1.669892384682166	6.392895459852728	6.632517623917536	1.4750172928966077	5.8325550775202775	6.282450358444395	1.3264399066178818	1.301613778143457	1.2772236917294648	5.91333234457842	6.1801350497052185	6.7079910480871625	6.480274870274416	1.4761705922212056	2.3714333181887426	5.520129753897521	6.1189421836137186	2.9514715291629816	3

18	1.4896605514900874	1.8612684443437302	3.3760515075852338	4.193193813972836	4.054083162694943	5.039038953995418	3.2907458471669377	3.19276886603068	3.466243975999634	3.193261775185116	2.7691713114390044	3.1785127499599284	2.039358334432588	4.433846008543861	3.6732611318311132	3.2775855159332545	3.0809759028491244	2.6098999383653	2.6426189477094617	4.113360803439548	3.6792222835265926	4.932060660537349	4.806471283306569	4.393701796528364	3.3480841952277984	3.036685763749003	4.724730702874903	5.977353514429787	6.066649029495363	5.315598677464747	5.3594059642536	4.720480059394697	4.562060184024101	2.9832052199809724	5.7068835847783905	3.3791407063846712	3.0796090395947475	4.8628161789676865	5.786136933804059	2.432891499117526	4.809573913919703	4.248725772001682	1.9981357419011556	2.6859511737518678	2.0491742573171523	4.595723511682201	5.310553243077953	6.272703797243557	5.179402869197414	2.7509255713052534	4.144233296416587	4.127529253403696	4.671042014897654	2.7491729645188894	3.404545

36	1.9833752278730148	2.193830355308627	3.654115723787887	3.7771717862929783	4.131607384881615	4.525555158920268	4.271618808830268	3.6934872146182247	4.041783070628382	3.8667663175029725	2.9188387090844787	2.891176468805928	2.1006192608348324	3.571782528148012	3.2408437959047007	4.135535391894198	2.537137182988316	3.302070044362956	2.604622694615819	4.4241007537791015	3.715339720358652	5.001464103307532	4.427952054283306	4.622908437399879	2.4921051144194086	2.6317260926484454	4.967150300625394	6.135428160599159	6.406908544927045	5.591097202882678	4.95875446661849	5.0903352170569685	4.378551057441155	2.4876739514328974	5.304994041342853	3.23209213436983	3.817621094395319	4.1059181352558545	5.352534164080231	1.8825590628226632	3.930478905353921	3.515384007782453	1.8726747180891177	2.4946097949544352	2.198719061880184	4.326915488751519	4.345157853624151	5.672468680557647	4.270695962247031	3.572528116756642	4.349223942126893	4.424620357636728	4.960260951294991	2.928834764498773	3.157653568

55	1.6933676890096205	1.9850048145859547	4.170838048787653	3.4848127141078904	3.104069225551336	4.209976241508222	4.2530469746511095	3.9084964149353945	4.278937180138959	4.063133527759922	3.36333673892169	2.8348297969352863	2.409224463855655	3.0799554422145414	2.6929569914609663	4.543301206833662	2.6712037943465567	3.229305808486867	2.8880059140337777	4.198465018687787	3.0552944106999957	5.655114063138377	4.091772683448816	4.640791309317362	2.059279591520819	3.320003808591072	4.82384474609959	6.060832615030547	6.362807718211053	5.576123944605622	5.485089745089888	5.121930530839213	4.0603373495419275	2.96378706397719	5.077841223792622	2.841322362188753	4.1971724070361836	3.9046864051705548	5.550381552140569	2.627449253653765	3.989337587810835	3.84984985428374	1.8384403389355652	2.182445988250379	2.369025424278351	4.234378498463009	4.224094445299104	5.969537905948554	3.7927978173554595	3.9119389173496475	4.574571692021781	4.107172781746377	4.843099108001992	3.16090665060521	3.08833531753

69	2.288161421338458	1.7931315085405257	4.120158170340933	3.5569425610407723	3.445764890358472	4.60056043878045	4.635720308632406	3.7572470250945846	4.183144176260599	3.904915399503608	3.3594847326389807	2.3922649647115435	2.5193072777970142	3.308777791193326	2.5334795315895007	3.993725489643031	2.1629854875285264	2.786546106205981	2.8836943299893223	4.172809807669362	3.5041178395435373	5.600170715028631	4.3961058588022635	4.433700226815756	2.506291822145344	3.1850585110088665	5.358374932009262	6.328710785327711	6.44445296762281	5.809559923793939	5.519577440486136	4.9806730328079585	4.678754094862946	3.4437284520005345	5.952209007253888	2.6687641400464823	3.941513730988154	4.286621414283463	5.885507927398327	2.7736210757128865	4.228532263127313	3.840293455676635	1.9514211491637317	1.7429770917103986	2.2137565071049963	4.503487162625796	4.462497382222297	5.791668747882086	4.416822207023583	3.7325751333573556	4.031784785089035	3.8257201832463847	4.724960353595151	2.6761019173186438	2.998

In [18]:
num_clusters = 8 #8 is somewhat arbitrary but works well
clustering = kmeans(num_clusters, init = 'k-means++').fit(data_array, weights)

In [19]:
ideologies = pd.DataFrame()
ideologies['questions'] = pos_value_answers

for idx, row in enumerate(clustering.cluster_centers_):
    ideologies[str(idx)] = row

In [20]:
dict = {}
for i in range(num_clusters):
    dict[i] = 0
    
for i in clustering.labels_:
    dict[i] += weights[i]

total = sum(dict.values())

for i in range(8):
    dict[i] = dict[i] / total

print(dict)

{0: 0.16398719982089568, 1: 0.18628226629181804, 2: 0.11964092203647485, 3: 0.09604993384329348, 4: 0.20963136130391966, 5: 0.03553150423540177, 6: 0.06209027008475843, 7: 0.12678654238343812}


In [21]:
def demo_stats(column, data_source):
    stats = []

    all_vals = [i for i in sorted(list(set(data_source[column]))) if i >= 0]
    for i in range(num_clusters):
        stats.append((i, {}))
    
    for i in range(num_clusters):
        for x in all_vals:
            stats[i][1][x] = 0
    
    for idx, cluster in enumerate(clustering.labels_):
        feature = data_source[column][idx]
        if(feature >= 0):
            stats[cluster][1][feature] += weights[idx]

    for i in stats:
        total = sum(i[1].values())
        print(i[0], end='\t')
        
        for x in i[1]:
            print("%.2f%%" %(i[1][x]/total * 100) ,end='\t')
        print()

In [22]:
for i in demographics:
    print(demographics[i])
    print(demo_stats(demographics[i],demo))
    print()

PRE: WHAT IS PRESENT RELIGION OF R
0	15.47%	10.93%	0.52%	0.10%	4.99%	0.17%	1.80%	0.04%	17.01%	16.11%	12.37%	20.49%	
1	12.12%	17.89%	3.05%	0.19%	0.50%	0.85%	0.86%	0.58%	1.89%	6.02%	28.74%	27.30%	
2	38.07%	23.83%	1.50%	2.82%	1.08%	0.18%	0.00%	0.13%	0.69%	2.57%	17.71%	11.41%	
3	23.80%	22.97%	1.22%	0.69%	5.25%	0.32%	0.79%	0.77%	6.36%	9.69%	12.52%	15.63%	
4	27.67%	20.03%	2.16%	1.81%	0.36%	0.00%	0.40%	0.11%	1.89%	1.92%	25.51%	18.15%	
5	8.82%	20.49%	4.89%	1.41%	1.36%	2.55%	0.39%	0.51%	2.69%	1.97%	29.03%	25.89%	
6	48.38%	17.52%	2.14%	2.10%	1.43%	0.22%	0.41%	0.00%	0.57%	1.06%	17.67%	8.50%	
7	21.31%	27.98%	1.40%	2.05%	1.92%	1.06%	0.77%	1.24%	3.09%	4.53%	15.67%	18.97%	
None

PRE: RELIGIOUS IDENTIFICATION
0	1.28%	7.35%	0.70%	16.45%	5.14%	16.14%	33.30%	19.64%	
1	4.93%	27.62%	1.97%	7.26%	6.78%	1.90%	29.97%	19.57%	
2	4.59%	51.88%	2.93%	3.04%	6.32%	1.85%	16.33%	13.05%	
3	2.19%	24.47%	2.40%	11.10%	5.02%	8.21%	30.01%	16.59%	
4	6.32%	45.02%	1.66%	2.52%	4.89%	0.97%	19.46%	19.15%	
5	8.09%	29.06%	3.55%	4.88

In [23]:
for i in media_sources:
    print(media_sources[i])
    print(demo_stats(media_sources[i],media))
    print()

PRE: MENTION: TV PROG - THE LATE SHOW WITH STEPHEN COLBERT
0	87.19%	12.81%	
1	84.58%	15.42%	
2	85.87%	14.13%	
3	79.14%	20.86%	
4	86.17%	13.83%	
5	86.90%	13.10%	
6	82.65%	17.35%	
7	82.99%	17.01%	
None

PRE: MENTION: TV PROG - TUCKER CARLSON TONIGHT (FOX)
0	85.92%	14.08%	
1	87.34%	12.66%	
2	86.12%	13.88%	
3	81.00%	19.00%	
4	88.85%	11.15%	
5	88.77%	11.23%	
6	85.29%	14.71%	
7	86.02%	13.98%	
None

PRE: MENTION: TV PROG - THE RACHEL MADDOW SHOW (MSNBC)
0	82.90%	17.10%	
1	87.16%	12.84%	
2	87.96%	12.04%	
3	78.82%	21.18%	
4	90.26%	9.74%	
5	85.58%	14.42%	
6	84.33%	15.67%	
7	86.86%	13.14%	
None

PRE: MENTION: TV PROG - THE LAST WORD WITH LAWRENCE O’DONNELL (MSNBC)
0	87.80%	12.20%	
1	88.93%	11.07%	
2	86.78%	13.22%	
3	80.06%	19.94%	
4	90.48%	9.52%	
5	87.95%	12.05%	
6	85.23%	14.77%	
7	86.53%	13.47%	
None

PRE: MENTION: TV PROG - THE FIVE (FOX)
0	85.14%	14.86%	
1	85.54%	14.46%	
2	85.75%	14.25%	
3	82.19%	17.81%	
4	88.54%	11.46%	
5	91.80%	8.20%	
6	86.49%	13.51%	
7	89.09%	10.91%	
None

PRE: MENTION: TV 

0	58.48%	41.52%	
1	65.22%	34.78%	
2	87.27%	12.73%	
3	56.72%	43.28%	
4	88.92%	11.08%	
5	78.49%	21.51%	
6	92.28%	7.72%	
7	70.29%	29.71%	
None

PRE: MENTION: WEBSITE - HUFFINGTON POST (WWW.HUFFINGTONPOST.COM)
0	71.29%	28.71%	
1	81.51%	18.49%	
2	92.67%	7.33%	
3	76.21%	23.79%	
4	95.44%	4.56%	
5	97.31%	2.69%	
6	96.10%	3.90%	
7	89.41%	10.59%	
None

PRE: MENTION: WEBSITE - NEW YORK TIMES (NYTIMES.COM)
0	40.22%	59.78%	
1	78.79%	21.21%	
2	89.98%	10.02%	
3	59.11%	40.89%	
4	91.91%	8.09%	
5	87.89%	12.11%	
6	94.42%	5.58%	
7	82.09%	17.91%	
None

PRE: MENTION: WEBSITE - FOX NEWS (WWW.FOXNEWS.COM)
0	95.19%	4.81%	
1	88.79%	11.21%	
2	62.28%	37.72%	
3	90.91%	9.09%	
4	73.57%	26.43%	
5	85.47%	14.53%	
6	53.58%	46.42%	
7	85.32%	14.68%	
None

PRE: MENTION: WEBSITE - WASHINGTON POST (WASHINGTONPOST.COM)
0	47.54%	52.46%	
1	80.36%	19.64%	
2	89.61%	10.39%	
3	65.66%	34.34%	
4	93.40%	6.60%	
5	92.97%	7.03%	
6	93.87%	6.13%	
7	85.80%	14.20%	
None

PRE: MENTION: WEBSITE - THE GUARDIAN (THEGUARDIAN.COM)
0	76.03%	23.97%	


In [24]:
from sklearn.decomposition import PCA
num_components = 5
pca_ideo = PCA(num_components, svd_solver='full')
pca_ideo.fit(data_array)

PCA(n_components=5, svd_solver='full')

In [25]:
print(pca_ideo.singular_values_)

[425.52719792 177.46113337 159.09791696 142.58035448 129.70081428]


In [26]:
print(pca_ideo.explained_variance_)

[21.87140913  3.80389586  3.05739186  2.45550882  2.03192429]


In [27]:
for i in range(8):
    print(i, pca_ideo.transform(np.array(ideologies[str(i)]).reshape(1, -1)))

0 [[-7.27208614 -0.14700079 -0.22141049  0.64989735  0.72980827]]
1 [[-2.73895839  1.89771655 -0.92416757 -0.29626804 -0.00557916]]
2 [[ 4.231311   -1.85749493 -0.2213594   0.43141955 -0.02081226]]
3 [[-4.171206   -1.53792166  0.10586202 -0.51178948 -0.25956348]]
4 [[ 3.76655655  1.44455523 -0.1973021  -0.75811447 -0.39927276]]
5 [[1.19400836 1.90692251 2.97289147 0.70797848 0.20250424]]
6 [[ 7.5364571  -0.43243078 -0.86687648  0.21068669  0.90159133]]
7 [[-0.2456987  -0.57158737 -0.11088056 -0.16147522 -0.75394555]]


In [28]:
for j in range(num_clusters):
    for idx, row in ideologies.iterrows():
        data = row[str(j)]
        ideologies[str(j)][idx] = data * stdevs[idx] + avgs[idx]

<ipython-input-28-76d749c03ca1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideologies[str(j)][idx] = data * stdevs[idx] + avgs[idx]


In [29]:
ideologies
#prints average answers for each ideological group's centroid

,questions,0,1,2,3,4,5,6,7
0,govt run for benefit of all people,1.369877,1.256760,2.447603,1.798782,2.086658,3.441377,1.697799,1.711459
1,govt doesn't waste much tax money,2.349406,1.581690,1.649860,2.503940,1.775790,3.125201,1.308511,2.091039
2,none in govt corrupt,3.864270,3.173678,4.136517,4.597731,3.661344,4.085218,3.614909,4.193043
3,people never can be trusted,3.214012,4.474203,3.212352,3.186214,4.071499,4.606701,3.623776,3.629287
4,elections don't make govt pay attention,3.478887,4.235547,3.193277,2.797954,3.715643,3.996287,3.591663,3.583477
5,govt provide many more services,6.050576,5.627890,2.951515,5.112896,4.481712,4.969463,2.646050,4.423161
6,more defense spending,2.399597,4.081290,5.057663,3.716761,5.207327,4.408466,5.474000,4.355445
7,private healthcare,1.746310,2.616812,5.640229,2.888478,4.385478,3.747774,5.961507,3.664241
8,govt shouldn't provide guaranteed jobs & ubi,2.250446,3.014593,5.823924,3.584468,4.537706,3.898029,6.225900,4.371390
9,govt shouldnt help black people,1.551816,2.541140,5.042712,2.836125,5.088713,3.677633,5.933168,4.070180


In [30]:
transformed = pca_ideo.transform(data_array)

In [31]:
def demo_ideos(column,data_source):
    stats = {}

    all_vals = [i for i in sorted(list(set(data_source[column]))) if i >= 0 and i <= 90] #all demographic groups
    for i in all_vals:
        stats[i] = [0 for i in range(num_components)] #calculate average loadings for each demographic group
    
    sums = {i:0 for i in all_vals}
   
    for idx, row in enumerate(transformed):
        demographic_group = data_source[column][idx]
        if(demographic_group not in all_vals):
            continue
        sums[demographic_group] += weights[idx]
        for component in range(num_components):
            stats[demographic_group][component] += row[component] * weights[idx]
        
    for idx1, val_sums in enumerate(stats):
        for idx2, comp_sum in enumerate(stats[val_sums]):
            stats[all_vals[idx1]][idx2] = comp_sum / sums[all_vals[idx1]]
            
    return stats
    
demo_ideos('PRE: SUMMARY: R SELF-IDENTIFIED RACE/ETHNICITY',demo)

#gives average ideology scores from pca for each demographic group

{1: [0.8304389498360115,
  -0.2369489881866511,
  -0.12382688949934596,
  0.1265431492462722,
  -0.08671056870797204],
 2: [-1.6746255618301333,
  1.526585154195292,
  0.5704239532736892,
  -0.6153599808814033,
  0.6742559931856958],
 3: [-0.8546331590819846,
  0.8644625756633013,
  0.36896118024925617,
  -0.04664891229947472,
  -0.02885377867158452],
 4: [-0.8464926035860197,
  0.22046538786739678,
  0.5189017806395458,
  0.15612634785007248,
  -0.3555814875513518],
 5: [0.5446843572652366,
  0.3889344502682897,
  0.033806046609911294,
  0.09376844265293284,
  0.01753223790950519],
 6: [-1.170766899099327,
  0.6735344958108394,
  -0.16559716134057811,
  0.34917083281138545,
  0.11616488739369953]}

In [32]:
def weighted_median(indices,component):
    val_weights = [(transformed[idx][component], weights[idx]) for idx in indices]
    sorted_items = sorted(val_weights, key=lambda x: x[0])
    
    sorted_vals = [i[0] for i in sorted_items]
    sorted_weights = [i[1] for i in sorted_items]
    
    cumsum = [sum(sorted_weights[:idx]) for idx in range(len(sorted_weights))]
    
    cutoff = sum(sorted_weights) / 2.
    
    for idx, value in enumerate(sorted_weights):
        if(cumsum[idx] >= cutoff):
            return sorted_vals[idx]
    print(sorted_vals, cumsum, cutoff)
    return None

def demo_medians(column,data_source):
    stats = {}

    all_vals = [i for i in sorted(list(set(data_source[column]))) if i >= 0 and i <= 90] #all demographic groups
    for i in all_vals:
        stats[i] = [0 for i in range(num_components)] #calculate average loadings for each demographic group
    
    sums = {i:0 for i in all_vals}
    
    for demographic_group in all_vals:
        for component in range(num_components):
            indices = [idx for idx in range(len(transformed)) if data_source[column][idx] == demographic_group]
            stats[demographic_group][component] = weighted_median(indices,component)
            #print(stats[demographic_group][component])
            
    return stats
    
demo_medians('PRE: SUMMARY: R SELF-IDENTIFIED RACE/ETHNICITY',demo)

#gives average ideology scores from pca for each demographic group

{1: [1.5422565266207464,
  -0.23652789808809388,
  -0.24806600733458636,
  0.11112315935793729,
  -0.22481416861911444],
 2: [-1.6682863942649133,
  1.5299620940211212,
  0.20573913647429035,
  -0.5015290539810927,
  0.5175825265699482],
 3: [-1.0025018981854354,
  0.8249288487076658,
  0.18117940234122315,
  0.048891027228959896,
  -0.1897860333798188],
 4: [-0.5734962698399876,
  0.2670783519899803,
  0.31465051093921076,
  0.2681720810933758,
  -0.41643862891436706],
 5: [0.387339064565167,
  0.5406530347245073,
  -0.4582255957924932,
  0.047967128317442556,
  -0.10857672561500388],
 6: [-1.6282411521216142,
  0.8008561850799115,
  -0.519519161364224,
  0.23687163190856805,
  0.03104716744650026]}

In [33]:
demo.head()

,PRE: WHAT IS PRESENT RELIGION OF R,PRE: RELIGIOUS IDENTIFICATION,PRE: MARITAL STATUS,PRE: SUMMARY: RESPONDENT 5 CATEGORY LEVEL OF EDUCATION,PRE: SEXUAL ORIENTATION OF R [REVISED],PRE: SUMMARY: TOTAL (FAMILY) INCOME,PRE-POST: SUMMARY: 2020 PRESIDENTIAL VOTE,PRE: WHAT IS YOUR (R) SEX? [REVISED],PRE: SUMMARY: R SELF-IDENTIFIED RACE/ETHNICITY,PRE: FOR WHICH CANDIDATE DID R VOTE IN PRESIDENTIAL PRIMARY,PRE: 7PT SCALE LIBERAL-CONSERVATIVE SELFPLACEMENT,Age
0,11,12,4,4,1,21,-1,1,3,8,6,46
1,12,13,1,3,1,13,3,2,4,10,4,37
2,11,8,1,2,1,17,1,2,1,6,2,40
3,2,5,1,3,1,7,1,1,4,-1,3,41
4,12,12,3,5,1,22,2,1,5,8,5,72


In [34]:
for i in demographics:
    print(demographics[i])
    x = demo_medians(demographics[i],demo)
    for i in x:
        print(i, end = '\t')
        for val in x[i]:
            print(val, end = '\t')
        print()
    print()

PRE: WHAT IS PRESENT RELIGION OF R
1	2.7871810048477013	-0.5174850237407268	-0.32400609908962213	-0.24461025724957478	0.40275560276500294	
2	0.35922721822651077	-0.01598954921326562	0.007672214017309681	-0.05807187606946308	-0.4557237276038913	
3	0.9954801884644034	0.9773624501199766	0.36943700493515275	0.2761827975134581	-0.1789921970306301	
4	2.8724740150497086	-0.6003182743747598	0.02149749095166707	-0.011555316094461462	0.2953019138282663	
5	-3.380640301251216	-0.7268525659753622	0.3266817472535625	0.06521653362657968	-0.4187660598604266	
6	0.27367337044025186	1.1822552530601045	0.7133487929769621	0.2025030523237162	1.058331621612184	
7	-3.77241048595542	0.34957626010320003	-0.40785181105050416	0.5797803467141022	0.024245505133911344	
8	-0.3476952918103839	0.32836842340327616	0.14381084961914103	-0.7470738159143674	-1.0739160447321343	
9	-5.120310811042939	0.12444427292987727	-0.21996880216415493	0.7341462314094166	-0.08000689453827285	
10	-3.9674819397774477	0.08541836740893126	-0

1	-6.741603845876719	0.7305146996327548	-0.3665342540882912	0.7015435793930139	0.69538620124983	
2	-5.534004035389965	-0.1313074850651999	-0.20800946240053328	0.24881314465683407	0.052933087067435595	
3	-3.36723471342537	-0.084946187917565	-0.1662438872471426	0.2690467720459437	-0.2820333389080269	
4	-0.5101937097789124	0.45059534431614456	-0.09041378261001962	-0.03186948068015048	-0.5604578735760252	
5	1.907701500821283	-0.33795417018244195	-0.10373584945791281	0.2968214857911983	-0.3881864547312009	
6	4.884960729603266	-0.6539556662865242	-0.42796797075711024	-0.08071310887544172	0.21734766503371605	
7	6.664320646674513	-0.46115086852953524	-0.5404009417425311	-0.3216711745680657	1.350523360327851	

Age
18	-1.2970194760141331	0.9447742915194173	-0.08910619929976059	1.567473214686667	0.20741301866686254	
19	-1.5441669889311456	0.9488261187554514	0.9969847909994888	0.5276502495487548	0.22611669179666996	
20	-0.6609097360364177	1.1719322807855883	0.7970793861738156	0.8516773867201415	0.

In [35]:
for i in media_sources:
    print(media_sources[i])
    x = demo_ideos(media_sources[i],media)
    for i in x:
        print(i, end = '\t')
        for val in x[i]:
            print(val, end = '\t')
        print()
    print()

PRE: MENTION: TV PROG - THE LATE SHOW WITH STEPHEN COLBERT
0	0.158949370613966	0.15987391472282458	-0.043387125373550106	0.012031701919916864	-0.028894164427173574	
1	0.1078565761586094	-0.01381866803938564	0.020845801957789054	-0.4765202001518397	-0.08560144889649408	

PRE: MENTION: TV PROG - TUCKER CARLSON TONIGHT (FOX)
0	0.18839426020971106	0.1638861184024419	-0.03516722763378902	-0.012672747402202648	-0.04699166541547224	
1	-0.08207848580617623	-0.06108050960324917	-0.022110777212625603	-0.38531100230527193	0.019838011924705884	

PRE: MENTION: TV PROG - THE RACHEL MADDOW SHOW (MSNBC)
0	0.23415764988895865	0.18031213494039433	-0.03370707482068052	-0.005690284079168089	-0.04604823140354085	
1	-0.34540125980230274	-0.15098202574367367	-0.031294908346924326	-0.4135328671867873	0.011797218923150705	

PRE: MENTION: TV PROG - THE LAST WORD WITH LAWRENCE O’DONNELL (MSNBC)
0	0.1929548223253115	0.16404919872648274	-0.023719989450329187	-0.016499864453796278	-0.025327016142586596	
1	-0.124923

0	0.98023133939294	0.018752218813749597	-0.20096608271683855	0.054044393395726795	-0.033180904314446986	
1	-4.0468477906638745	-0.8563893100363624	-0.18294417137857982	0.2778656310843033	0.25051746859378937	

PRE: MENTION: RADIO PROG - THE DAVE RAMSEY SHOW
0	-0.09267487084670184	-0.07829212666860887	-0.19682092923876002	0.08646931151904891	-0.0016198797595677098	
1	4.014784938706135	-0.7868732310496538	-0.21628687301670513	0.14703093055000196	0.2264697025336915	

PRE: MENTION: RADIO PROG - MORNING EDITION (NPR)
0	0.877094792469656	-0.021721659247197524	-0.22498507040595767	0.06396272197982103	-0.01946831716699144	
1	-4.4480435505350115	-0.7800589393860461	-0.023198721828986125	0.26115467668033815	0.22193047504031038	

PRE: MENTION: RADIO PROG - THE MARK LEVIN SHOW
0	-0.18663762929367034	-0.05805603350402479	-0.15921468597090102	0.09139780228390651	-0.022459072587311776	
1	6.120938702491544	-1.2181569084416477	-0.8533792899856236	0.07153410458534702	0.6070450115519043	

PRE: MENTION: RA

0	0.058348078259723324	-0.11204326603088884	-0.11248196161977135	-0.030139214896682354	-0.08712071948937812	
1	-4.353257672081502	-0.5348470127414967	-0.11356721714533641	0.2167048036035843	0.3179402620104366	

